In [1]:
!git clone https://github.com/Oliver-Lai/monopoly

fatal: destination path 'monopoly' already exists and is not an empty directory.


# 產生訓練及測試資料
#### 輸入：[玩家初始金額(500～2500)，監獄罰金(35～150)]
#### 輸出：遊玩2000次的平均回合數

In [2]:
import numpy as np
import random as rd
from monopoly.playground import monopoly_with_fair_prison

rd.seed(42)
NUM_SIMULATIONS = 2000
# 固定數值
GAME_MAP = [rd.randint(400,500) for i in range(10)]
PLAYER_NUM = 4
PRISON_LOCATION = 5
PRISON_DAY = 2

# 輸入變動數值
# PLAYERS_MONEY
# PRISON_FINE

## 產生3000筆資料

In [3]:
input_data = np.array([[rd.randint(500,2500),rd.randint(35,150)] for i in range(3000)])
input_data

array([[1885,  129],
       [2327,  104],
       [ 678,  110],
       ...,
       [ 685,   71],
       [ 556,   84],
       [ 601,   55]])

### 開始執行模擬3000場大富翁(大概花費26分鐘)

In [4]:
output = []
for i in input_data:
    turn = []
    PLAYERS_MONEY = i[0]
    PRISON_FINE = i[1]
    for j in range(NUM_SIMULATIONS):
        game = monopoly_with_fair_prison(PLAYERS_MONEY,PLAYER_NUM,GAME_MAP,PRISON_LOCATION,PRISON_DAY,j,PRISON_FINE)
        game.play_game()
        turn.append(game.get_turn())
    output.append(round(np.mean(turn),2))

In [5]:
output = np.array(output)

In [6]:
output

array([43.44, 81.67, 19.41, ..., 23.18, 19.04, 24.1 ])

# 訓練MLP模型

In [7]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(input_data,output ,
                                   random_state=104,
                                   test_size=0.3,
                                   shuffle=True)

## 資料做標準化

In [8]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
X_train_scaled

array([[-0.1172308 , -0.69245664],
       [ 1.08281912,  0.20886293],
       [ 0.3162674 , -0.84267657],
       ...,
       [ 1.61147547,  1.02005054],
       [-0.09960893,  0.74965467],
       [ 1.54451233,  0.74965467]])

## 設定MLP架構並訓練

In [10]:
model = MLPRegressor(solver="adam",activation="relu",max_iter=20000,hidden_layer_sizes=[64,32], random_state=42)

In [11]:
model.fit(X_train_scaled,y_train)

MLPRegressor(hidden_layer_sizes=[64, 32], max_iter=20000, random_state=42)

In [12]:
y_pred = model.predict(X_train_scaled)

## 在訓練資料中的準確度

In [13]:
from sklearn.metrics import root_mean_squared_error
root_mean_squared_error(y_train,y_pred)

0.7017071100120144

In [14]:
y_pred = model.predict(X_test_scaled)

## 在測試資料中的準確度

In [15]:
root_mean_squared_error(y_test,y_pred)

0.7197616328258057

# 儲存模型、Scaler及輸入輸出資料

In [16]:
import joblib
joblib.dump(model,"model.pkl")
joblib.dump(scaler, 'scaler.save')

['scaler.save']

In [17]:
scaler_2 = joblib.load('scaler.save')
model_2 = joblib.load("model.pkl")
test = scaler_2.transform([[1000,75]])
model_2.predict(test)

array([20.35503884])

In [18]:
np.save('input_data.npy',input_data)
np.save('output.npy',output)